# Introduction
## SBA - Small Business Profiles for the States and Territories

The Office of Advocacy’s Small Business Profiles are an annual analysis of each state’s small business activities. Each profile gathers the latest information from key federal data-gathering agencies to provide a snapshot of small business health and economic activity. This year’s profiles report on state economic growth and employment; small business employment, industry composition, and turnover; plus business owner demographics and county-level employment change. 

https://www.sba.gov/

In [1]:
from IPython.core.display import display, HTML
display(HTML("""<style> .container {width:96% !important;}</style>"""))

from IPython.display import IFrame

In [2]:
import pandas as pd
import multiprocessing
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial
import math

# Handle s3 or local
import s3fs
from os import listdir
from os.path import isfile, join
import subprocess

## Path to the files

In [3]:
import sys
sys.path.insert(0,'../')
from Tools.paths import *

In [4]:
def list_files(path,ext = 'pdf'):
    if path.startswith('s3://'):  
        onlyfiles = subprocess.check_output(['aws', 's3', 'ls', path_s3])
        onlyfiles = onlyfiles.split('\n')
        onlyfiles = [f.split(" ")[-1] for f in onlyfiles]
    else:
        onlyfiles = [f for f in listdir(path_local) if isfile(join(path_local, f))]
    onlyfiles = [f for f in onlyfiles if f.endswith('.{}'.format(ext))]
    files = [f.replace('.{}'.format(ext),'') for f in onlyfiles]
    return files

In [5]:
def path(path,name,ext = 'pdf'):
    path_file = '{}{}.{}'.format(path,name,ext)
    return path_file

## Loading the file with pdf_query

PDFQuery is a light wrapper around pdfminer, lxml and pyquery. It's designed to reliably extract data from sets of PDFs with as little code as possible.

In [6]:
import pdfquery

In [7]:
def load_pdf(path_file):
    if path_file.startswith('s3://'):  
        fs = s3fs.S3FileSystem()
        with fs.open(path_file, 'rb') as fp_in:
            pdf = pdfquery.PDFQuery(fp_in)
            pdf.load()
    else:
        pdf = pdfquery.PDFQuery(path_file)
        pdf.load()        
    return pdf

In [8]:
def getCoordinates(pdf,query, type_search = "Line"):
        name = pdf.pq('LTText%sHorizontal:contains("%s")' % (type_search,query))
        for n in name:
            d = dict()
            d["left_corner"] = math.floor(float(n.layout.x0)* 1000)/1000.0
            d["bottom_corner"] = math.floor(float(n.layout.y0)* 1000)/1000.0
            d["right_corner"] = math.ceil(float(n.layout.x1)* 1000)/1000.0
            d["upper_corner"] = math.ceil(float(n.layout.y1)* 1000)/1000.0
            d["text"] = n.layout.get_text()
            d["pageid"] = int(float(n.iterancestors('LTPage').next().layout.pageid))
            yield d

In [9]:
def getTable(file_pdf, col_width, row_space, row_height,title,bottom_corner_dif,headers,col_left_align):
    
    table = list()
    table.append(headers)
    
    g = getCoordinates(pdf=file_pdf,query=title,type_search="Line")
    d = next(g,None)
    
    pageid = d['pageid']
    bottom_corner = d['bottom_corner'] - bottom_corner_dif

    while 1:
        columns = (c for c in xrange(len(headers)))
        boxes = list()
        for c in columns:
            boxes.append(tuple(('col_%s' %(c),
                               'LTPage[pageid="%s"] LTTextLineHorizontal:overlaps_bbox("%f,%f,%f,%f")' % (pageid,
                                                                                                          col_left_align[c],
                                                                                                          bottom_corner,
                                                                                                          col_left_align[c]+col_width,
                                                                                                          bottom_corner+row_height))))



        columns = [c for c in xrange(len(headers))]
        row = file_pdf.extract(boxes)
        columns = [row['col_{}'.format(c)].text() for c in columns]
        table.append(columns)
        if 'Total' in row['col_0'].text():
            break

        bottom_corner -= row_space
    return table

In [10]:
def info1(file_pdf):
    col_right_align = 300
    DemographicGroup = ['American-owned',
                        'Asian-owned',
                        'Islander-owned',
                        'Hispanic-owned',
                        'Alaskan-owned',
                        'Minority-owned',
                        'Nonminority-owned']    
    
    DemographicInfo = [('with_formatter', 'text')]
    
    for dg in DemographicGroup:
        g = getCoordinates(pdf=file_pdf,query=dg,type_search="Line")
        d = next(g,None)
        DemographicInfo.append(tuple((dg,'LTTextLineHorizontal:in_bbox("%f,%f,%f,%f")'%(d["left_corner"],
                                                                                        d["bottom_corner"],
                                                                                        col_right_align,
                                                                                        d["upper_corner"]))))
    info = file_pdf.extract(DemographicInfo)
    return info

In [11]:
def info2(file_pdf):
    col_width = 35
    col_left_align = [50,295,371,449,532]
    row_space = 16.78
    row_height = 14
    bottom_corner_dif = 126.91
    headers = ['Industry',
                '1-499 Employees',
                '1-19 Employees',
                'Nonemployer Firms',
                'Total Small Firms'] 

    table = getTable(col_left_align=col_left_align,
                     col_width=col_width,
                     file_pdf=file_pdf,
                     headers=headers,
                     row_height=row_height,
                     row_space = row_space,
                     bottom_corner_dif=bottom_corner_dif,
                     title = "Table 1")
    
    table.pop(0)
    df = pd.DataFrame(table,columns=headers)
                     
    return df

In [12]:
def info3(file_pdf):
    col_width = 35
    col_left_align = [50,325,400,532]
    row_space = 13.6
    row_height = 12.4
    bottom_corner_dif = 115.5

    headers = ['Industry',
               'Small Business Employment',
               'Total Private Employment',
               'Small Business Emp Share']    
    
    table = getTable(col_left_align=col_left_align,
                     col_width=col_width,
                     file_pdf=file_pdf,
                     headers=headers,
                     row_height=row_height,
                     row_space = row_space,
                     bottom_corner_dif=bottom_corner_dif,
                     title = "Table 2")
    table.pop(0)
    df = pd.DataFrame(table,columns=headers)

    return df

### How about several pdf's at the same time?

In [13]:
def process_file(path_file):
    file_pdf = load_pdf(path_file)
    d = dict()
    d['file'] = path_file.split('/')[-1]
    d.update(info1(file_pdf))
    x = info2(file_pdf)
    d['industry'] = x
    x = info3(file_pdf)
    d['employment'] = x
    return d

In [14]:
# https://stackoverflow.com/questions/29494001/how-can-i-abort-a-task-in-a-multiprocessing-pool-after-a-timeout
def abortable_worker(func, *args, **kwargs):
    timeout = kwargs.get('timeout', None)
    p = ThreadPool(1)
    res = p.apply_async(func, args=args)
    try:
        out = res.get(timeout)  # Wait timeout seconds for func to complete.
        return out
    except multiprocessing.TimeoutError:
        print("Aborting due to timeout ")
        p.terminate()
        raise

In [15]:
%%time
if __name__ == '__main__':    
    result = list()
    pool = multiprocessing.Pool(maxtasksperchild=1)
    files = list_files(path_s3)
    files = files[0:5]
    for i in files:
        abortable_func = partial(abortable_worker, process_file, timeout=60)
        path_file = path(path_s3,i)
        pool.apply_async(abortable_func, args=(path_file, ), callback=result.append)
        print i
    pool.close()
    pool.join()
    print result[0]

Alabama
Alaska
American_Samoa
Arizona
Arkansas
{'Nonminority-owned': 'Nonminority-owned l -8.6%', 'Minority-owned': 'Minority-owned l 30.7%', 'Alaskan-owned': 'Native American/Alaskan-owned l 27.0%', 'Hispanic-owned': 'Hispanic-owned l 51.5%', 'Islander-owned': u'Hawaiian/Paci\ufb01c Islander-owned l -16.9%', 'industry':                                            Industry 1-499 Employees  \
0                                      Retail Trade          10,674   
1     Other Services (except Public Administration)          10,042   
2   Professional, Scientiﬁc, and Technical Services           8,081   
3                 Health Care and Social Assistance           7,823   
4                                      Construction           7,143   
5                   Accommodation and Food Services           5,525   
6                                   Wholesale Trade           3,785   
7                                     Manufacturing           3,377   
8     Administrative, Support, and Was